In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud ,STOPWORDS
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize 

import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation,Flatten
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/RelevantTweet_New_03_1_23.csv')

In [6]:
df

,account,ioc_value,text,isValid
0,neilnews91,http://insidethegames.biz,In the last hour #Brisbane has been officially...,0
1,SRMInform,https://hubs.li/H0T6XWG0,.@BBCRadio4 recently interviewed S-RM's Head o...,0
2,dnstwist,http://g00qle.com,g00qle[.]com #phishing,1
3,HeliosCert,179.108.82.211,Sample submitted\n2021-02-02 00:05:02\nDionaea...,1
4,ecarlesi,http://dtcecommercecommunity.com/,Threat on hxxp://dtcecommercecommunity[.]com/ ...,1
...,...,...,...,...
32602,AvsTerritory,https://www.espn.com/nhl/story/_/id/31843898,The 21-22 #NHL schedule that will be released ...,0
32603,HeliosCert,2c5ff4efee54c88d97a91a5651802534ec126d44dbc384...,Sample submitted\n2021-05-07 17:05:02\nDionaea...,1
32604,lgomezperu,https://lnkd.in/dgxvgeV,@TheCyberSecHub\n#Scam artists exploit Kaseya ...,0
32605,KesaGataMe0,172.245.184.102,#Phishing #JACCS #JACCSCARD #ジャックスカード \n\nIP:1...,1


In [7]:
df['isValid'].value_counts()

0    20200
1    12407
Name: isValid, dtype: int64

In [8]:
#Remove null values
df = df.dropna(axis=0, subset=['text'])

In [9]:
#Remove null values
df=df.drop_duplicates(subset='text', keep="first")

In [10]:
df['isValid'].value_counts()

0    20188
1    12400
Name: isValid, dtype: int64

In [11]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [12]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [13]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Preprocessing

In [15]:
def replaceDegfanged(text):
    """ Replaces url address with "url" """
    text = re.sub('([A-Za-z]+://)([-\w]+(?:\.\w[-\w]*)+)(:\d+)?(/[^.!,?;"\x27<>()\[\]{}\s\x7F-\xFF]*(?:[.!,?]+[^.!,?;"\x27<>()\[\]{}\s\x7F-\xFF]+)*)?','[defanged]',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

In [16]:
def replaceURL(text):
    """ Replaces url address with "url" """
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','[url]',text)
    text = re.sub(r'#([^\s]+)', r'\1', text)
    return text

In [17]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [18]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('A'):
        return wordnet.ADJ
    elif treebank_tag.startswith('S'):
        return wordnet.ADJ_SAT
    elif treebank_tag.startswith('SYM'):
        return wordnet.SYM
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV    
    elif treebank_tag.startswith('WRB'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [19]:
#@title
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [20]:
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()

https://www.kaggle.com/code/christofhenkel/how-to-preprocessing-for-glove-part1-eda/notebook
https://www.kaggle.com/code/sudalairajkumar/getting-started-with-text-preprocessing/notebook



In [21]:

def preprocessing_df(text):
  tweets = []
  for sentence in tqdm(text):
    sentence = sentence.lower() # converting the words to lower case
    sentence = re.sub("@[A-Za-z0-9_]+","", sentence) # removing the words starts with "@"
    #sentence = re.sub("#[A-Za-z0-9_]+","", sentence) # removing the words starts with "H"
    sentence = re.sub(r'https://t.co/\S+','', sentence)
    sentence =  replaceURL(sentence) # replace the url from the sentence
    sentence = re.sub(r'hxxp\S+','[defanged]', sentence)
    sentence = replaceDegfanged(sentence)
    sentence = re.sub('CVE-\d{4}-\d{4,7}','[cve]',sentence)
    sentence = sentence.replace('dot', '.')
    sentence = sentence.replace('[.]', '.')
    sentence = re.sub(r"[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}", "[ip]",sentence)
    sentence = re.sub(r"\b^\$2[ayb]\$.{64}$\b",'[hash]',sentence) #hash 512
    sentence = re.sub(r"\b^\$2[ayb]\$.{56}$\b",'[hash]',sentence) #hash 256
    sentence = re.sub(r"\b[a-f0-9]{40}|[A-F0-9]{40}\b",'[hash]',sentence) #hash 1
    sentence = re.sub(r"\b[a-f0-9]{32}|[A-F0-9]{32}\b",'[hash]',sentence) #md5
    sentence = re.sub(r"\b[a-z0-9]+@[a-z]+\.[a-z]{2,3}\b","",sentence)
    sentence = remove_emoji(sentence)
    sentence = chat_words_conversion(sentence)
    #sentence = correct_spellings(sentence)
    sentence = re.sub("(?<=\.)([^.]+)(?:\.(?:co\.uk|ac\.us|com|net|org|info|coop|int|co|ac|ie|co|ai|eu|ca|icu|top|xyz|tk|cn|ga|cf|nl|us|eu|de|hk|am|tv|bingo|blackfriday|gov|edu|mil|arpa|au|ru|[^.]+(?:$|\n)))",'[domain]',sentence)
    sentence = re.sub("[^a-zA-Z\[\] ']", "", sentence) # removing symbols
    sentence = ' '.join( [w for w in sentence.split() if len(w)>1] )
    sentence = sentence.split()
    word_and_tag = nltk.pos_tag(sentence)
    sentence = [wl.lemmatize(word,pos=get_wordnet_pos(tag)) for word,tag in word_and_tag if word not in set(stopwords.words("english"))]
    sentence = " ".join(sentence)
    tweets.append(sentence)
  return tweets
df['preprocess_text'] = preprocessing_df(df['text'])

100%|██████████| 32588/32588 [01:38<00:00, 332.51it/s]


In [22]:
# shuffle the DataFrame rows
df = df.sample(frac = 1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22600 entries, 1566 to 13561
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   account          22600 non-null  object
 1   ioc_value        22600 non-null  object
 2   text             22600 non-null  object
 3   isValid          22600 non-null  int64 
 4   preprocess_text  22600 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


In [23]:
df['isValid'].value_counts()

0    20188
1    12400
Name: isValid, dtype: int64

In [24]:
#Remove null values
df = df.dropna(axis=0, subset=['preprocess_text'])

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32588 entries, 17618 to 12066
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   account          32588 non-null  object
 1   ioc_value        32588 non-null  object
 2   text             32588 non-null  object
 3   isValid          32588 non-null  int64 
 4   preprocess_text  32588 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.5+ MB


In [26]:
df.to_csv( "/content/drive/MyDrive/Twitter_Work_Code/dataset/Preprocessed_Tweet_DF_new_3_1_2023.csv", index=False)

In [27]:
t = pd.read_csv("/content/drive/MyDrive/Twitter_Work_Code/dataset/Preprocessed_Tweet_DF_new_3_1_2023.csv")

In [28]:
test=t[t['preprocess_text'].isna()]

In [ ]:
test

,account,ioc_value,text,isValid,preprocess_text
6635,hasfarr,https://pastebin.com/GjV6mzHS,It's up https://t.co/pIeM9WpYz6 https://t.co/o...,0,NaN
9352,BennyHummm,https://pastebin.com/Rs5U79W4,@llsilxnt https://t.co/hOumSwEDLU\n\nthis?,0,NaN
11123,stardustsweep,https://pastebin.com/raw/JQA2FEJD,@sleepysowikus https://t.co/f5lQt4jsU5 this is...,0,NaN
19748,James_inthe_box,https://www.hybrid-analysis.com/string-search/...,And a few more:\n\nhttps://t.co/mdmTlpFpJd,0,NaN
21046,sumiitaru,https://pastebin.com/dE0Whv4K,@owenchanlovebot its here 🤲https://t.co/D2XXTi...,0,NaN


In [ ]:
a=df.iloc[21046]

In [ ]:
a

account                                                    sumiitaru
ioc_value                              https://pastebin.com/dE0Whv4K
text               @owenchanlovebot its here 🤲https://t.co/D2XXTi...
isValid                                                            0
preprocess_text                                                     
Name: 16039, dtype: object

In [ ]:
type(a.preprocess_text)

str

In [29]:
t['isValid'].value_counts()

0    20188
1    12400
Name: isValid, dtype: int64

In [ ]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22600 entries, 0 to 22599
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   account          22600 non-null  object
 1   ioc_value        22600 non-null  object
 2   text             22600 non-null  object
 3   isValid          22600 non-null  int64 
 4   preprocess_text  22595 non-null  object
dtypes: int64(1), object(4)
memory usage: 882.9+ KB


In [ ]:
#Remove null values
df=df.drop_duplicates(subset='preprocess_text', keep="first")

In [ ]:
df

,account,ioc_value,text,isValid,preprocess_text
12432,mcgallen,http://microwire.info,The latest briefing @ https://t.co/I22ExmNx1h!...,0,late briefing cybersecurity ransomware
20848,bennel62,https://paulsnewsline.blogspot.com/2021/07/get...,#GETMEREWRITE #IOC continues #boogyingitswayto...,0,getmerewrite ioc continue boogyingitswaytodoom...
11026,WiretapLawyer,https://www.theguardian.com/news/video/2021/ju...,Pegasus: the spyware technology that threatens...,0,pegasus spyware technology threaten democracy ...
5981,prolucid,https://thehackernews.com/2021/07/revil-ransom...,What happened to the #REvil #Ransomware gang? ...,0,happen revil ransomware gang cybersecurity dat...
13161,HeliosCert,0bf04dc073df0b29086ad48f719ef1aa7790f3a65693f3...,Sample submitted\n2021-05-06 17:55:02\nDionaea...,1,sample submit dionaea honeypot protocol smbd s...
...,...,...,...,...,...
3435,docmentillo,https://news.softpedia.com/news/israel-launche...,Israel Launches Anti-Ransomware Campaign to Pr...,0,israel launch antiransomware campaign prevent ...
21423,pivotpointsec,https://ecs.page.link/pTn51,Why It Now Takes 315 Days to Contain a Malicio...,0,take day contain malicious cyber attack infose...
15359,romonlyht,219.94.163.86,https://t.co/3yeTMA2VZw\n\nNo.1 (NXDOMAIN)\n\n...,1,nxdomain receive [ip] [defanged] phishing micard
19992,theCUBE365,https://video.cube365.net/c/933051,NETSCOUT helps enterprises with business conti...,0,netscout help enterprises business continuity ...


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/Preprocessed_Tweet_DF_new_11_11_2022.csv')

In [ ]:
#Remove null values
df = df.dropna(axis=0, subset=['preprocess_text'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline


tfidf = TfidfTransformer()
clf = CountVectorizer()

X_cv =  clf.fit_transform(df['preprocess_text'])

tf_transformer = TfidfTransformer(use_idf=True).fit(X_cv)
X_tf = tf_transformer.transform(X_cv)

In [ ]:
from sklearn.model_selection import train_test_split
# train and test
X_train, X_test, y_train, y_test = train_test_split(X_tf, df['isValid'], test_size=0.20, stratify=df['isValid'], random_state=42)

In [ ]:
y_train.value_counts()

0    8155
1    6312
Name: isValid, dtype: int64

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
rf_pred = rf_clf.predict(X_test)
print('Classification Report for Random Forest:\n',classification_report(y_test, rf_pred))

Classification Report for Random Forest:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      2039
           1       0.99      0.96      0.98      1578

    accuracy                           0.98      3617
   macro avg       0.98      0.98      0.98      3617
weighted avg       0.98      0.98      0.98      3617



In [ ]:
print('Confusion matrix  Report for Random Forest:\n',confusion_matrix(y_test, rf_pred))

Confusion matrix  Report for Random Forest:
 [[2030    9]
 [  62 1516]]


In [ ]:
df[100:150]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['preprocess_text'], 
                                                    df['isValid'], 
                                                    test_size=0.2, 
                                                    random_state=8)

In [ ]:
ngram_range = (1,3)
max_features = 3000

In [ ]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model  = RandomForestClassifier(random_state=1)
model.fit(features_train, labels_train)
model_predictions = model.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, model_predictions))
print(classification_report(labels_test, model_predictions))

In [ ]:
model = LogisticRegression()
print(model.get_params())
model.fit(features_train, labels_train)
model_predictions = model.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, model_predictions))
print(classification_report(labels_test, model_predictions))

{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Accuracy:  0.9830715888231695
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      4515
           1       0.98      0.81      0.88       388

    accuracy                           0.98      4903
   macro avg       0.98      0.90      0.94      4903
weighted avg       0.98      0.98      0.98      4903



In [ ]:

model = KNeighborsClassifier()
model.fit(features_train, labels_train)
model_predictions = model.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, model_predictions))
print(classification_report(labels_test, model_predictions))

Accuracy:  0.9787884968386702
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      4515
           1       0.95      0.78      0.85       388

    accuracy                           0.98      4903
   macro avg       0.96      0.89      0.92      4903
weighted avg       0.98      0.98      0.98      4903



In [ ]:
model = DecisionTreeClassifier(random_state=1)
model.fit(features_train, labels_train)
model_predictions = model.predict(features_test)
print('Accuracy: ', accuracy_score(labels_test, model_predictions))
print(classification_report(labels_test, model_predictions))

Accuracy:  0.9775647562716704
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4515
           1       0.88      0.83      0.85       388

    accuracy                           0.98      4903
   macro avg       0.93      0.91      0.92      4903
weighted avg       0.98      0.98      0.98      4903



In [ ]:

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []

    
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split(), [label]))
    
    return labeled


X_train, X_test, y_train, y_test = train_test_split(df.preprocess_text, df.isValid, random_state=0, test_size=0.2)

X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 18089/18089 [00:00<00:00, 2371850.85it/s]


In [ ]:

import numpy as np
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model = model.fit(train_vectors_dbow, y_train)
model_prediction = model.predict(test_vectors_dbow)
print('accuracy %s' % accuracy_score(model_prediction, y_test))
print(classification_report(y_test, model_prediction))

accuracy 0.9778883360972913
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2021
           1       0.99      0.96      0.97      1597

    accuracy                           0.98      3618
   macro avg       0.98      0.98      0.98      3618
weighted avg       0.98      0.98      0.98      3618



In [ ]:
from sklearn.model_selection import GridSearchCV
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(model, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(features_train, labels_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
# Tf-Idf (advanced variant of BoW)
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,2))

In [ ]:
corpus = X_train["preprocess_text"]
vectorizer.fit(corpus)
X_train = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_

In [ ]:
#@title
def create_wordcloud(words):
    wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(words)
    plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()

In [ ]:
#@title
subset=df[df.isValid==0]
text=subset.preprocess_text.values
words =" ".join(text)
create_wordcloud(words)

In [ ]:
#@title
subset=df[df.isValid==1]
text=subset.preprocess_text.values
words =" ".join(text)
create_wordcloud(words)

In [ ]:
#@title
def get_corpus(text):
    words = []
    for i in text:
        for j in i.split():
            words.append(j.strip())
    return words
corpus = get_corpus(df.preprocess_text)
corpus[:5]

In [ ]:
#@title
from collections import Counter
counter = Counter(corpus)
most_common = counter.most_common(10)
most_common = dict(most_common)
most_common

In [ ]:
#@title
from sklearn.feature_extraction.text import CountVectorizer
def get_top_text_ngrams(corpus, n, g):
    vec = CountVectorizer(ngram_range=(g, g)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
#@title
plt.figure(figsize = (16,9))
most_common_uni = get_top_text_ngrams(df.preprocess_text,10,1)
most_common_uni = dict(most_common_uni)
sns.barplot(x=list(most_common_uni.values()),y=list(most_common_uni.keys()))

In [ ]:
#@title
plt.figure(figsize = (16,9))
most_common_bi = get_top_text_ngrams(df.preprocess_text,10,2)
most_common_bi = dict(most_common_bi)
sns.barplot(x=list(most_common_bi.values()),y=list(most_common_bi.keys()))

In [ ]:
#@title
plt.figure(figsize = (16,9))
most_common_tri = get_top_text_ngrams(df.preprocess_text,10,3)
most_common_tri = dict(most_common_tri)
sns.barplot(x=list(most_common_tri.values()),y=list(most_common_tri.keys()))

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2022-10-22 10:10:30--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-10-22 10:10:30--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.01MB/s    in 2m 39s  

2022-10-22 10:13:09 (5.18 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [ ]:
!wget https://nlp.stanford.edu/data/glove.42B.300d.zip

--2022-10-20 06:41:38--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-10-20 06:41:39--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Saving to: ‘glove.42B.300d.zip’

glove.42B.300d.zip  100%[===================>]   1.75G  5.01MB/s    in 5m 52s  

2022-10-20 06:47:32 (5.08 MB/s) - ‘glove.42B.300d.zip’ saved [1877800501/1877800501]



In [ ]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
EMBEDDING_FILE = 'glove.6B.300d.txt'

https://www.kaggle.com/code/madz2000/nlp-using-glove-embeddings-99-87-accuracy/notebook

https://www.kaggle.com/code/hamishdickson/training-and-plotting-word2vec-with-bigrams

https://www.kaggle.com/code/jagannathrk/word2vec-cnn-text-classification

https://www.h2kinfosys.com/blog/word-embeddings-with-word2vec-tutorial-all-you-need-to-know/

https://nlp.stanford.edu/projects/glove/

In [ ]:
# prepare tokenizer
token = Tokenizer()
token.fit_on_texts(df['preprocess_text'])
max_len= max(len(s) for s in df['preprocess_text'])
vocab_size = len(token.word_index) + 1
# integer encode the documents
text = token.texts_to_sequences(df['preprocess_text'])
print(text)

[[944, 114, 307, 7, 23, 43, 1348, 173, 20, 814, 415, 564], [16, 20, 7, 364, 1404], [314, 146, 497, 30, 259, 420, 777, 1, 4, 4343, 35, 452, 34, 21, 154, 79, 3, 6, 2, 8, 9, 14, 14], [934, 1, 99, 676, 191, 373, 62, 23, 843, 591, 787, 83, 12030, 188, 2, 55, 36, 235, 1, 3, 29], [169, 16, 20, 7], [514, 69, 362, 1697], [16, 20, 7, 4344], [145, 31, 20, 31, 216, 7, 289], [422, 4890, 5598], [158, 165, 217, 515, 86, 799, 227, 11, 2486, 2, 484, 369, 1405, 677, 369, 1, 32, 2], [21, 5, 1210, 1, 5, 130, 597, 21, 4, 2], [216, 20, 7], [1349, 3336, 40, 388, 86, 1892, 12031], [736, 13, 1237, 8357, 411, 19, 412, 315, 13], [169, 16, 20, 7, 1271], [11, 43, 2487, 6667, 312, 8358, 109, 12032, 2, 1, 3, 8], [16, 20, 7, 4345, 1893], [334, 16, 283, 170, 23, 88, 1406, 84, 199, 10, 560, 170, 189, 104, 5, 1, 7, 155, 15], [240, 12033, 1407, 333, 12034, 2113, 2, 4, 133, 9, 12, 737, 5, 974, 3, 1, 1471, 168, 8, 423], [91, 989, 5, 489, 3337, 2622, 8, 6, 2, 29, 24, 4, 12, 310, 161, 9, 3, 349, 104, 67, 1, 33, 44, 93], [891

In [ ]:
MAX_SEQUENCE_LENGTH = 128
max_len=128
MAX_NB_WORDS = 10000
EMBEDDING_DIM=300
VALIDATION_SPLIT = 0.1
traina = True #embeedding layer trainable

In [ ]:
# pad documents to a max length of 4 words
text = pad_sequences(text, maxlen=max_len, padding='post')
print(text)

[[  944   114   307 ...     0     0     0]
 [   16    20     7 ...     0     0     0]
 [  314   146   497 ...     0     0     0]
 ...
 [28467    31    65 ...     0     0     0]
 [   65    60  3967 ...     0     0     0]
 [   13   157    35 ...     0     0     0]]


In [ ]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in token.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [ ]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['isValid'])
y = to_categorical(y)
y[:2]

array([[0., 1.],
       [0., 1.]], dtype=float32)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(text), y, test_size=0.2, stratify=y,random_state=100)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, stratify=y_test)

In [ ]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 26.5 MB/s 
     |████████████████████████████████| 1.6 MB 52.8 MB/s 


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
def build_model(hp):
    keras_model = keras.Sequential()
    #keras_model.add(word2vec_model.wv.get_keras_embedding(True))
    keras_model.add(Embedding(vocab_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=traina))
    keras_model.add(Dropout(0.5))
    keras_model.add(Conv1D(filters=hp.Int('filters',
                                        min_value=32,
                                        max_value=512,
                                        step = 32),
                kernel_size=hp.Int('kernel_size',
                                        min_value=3,
                                        max_value=11,
                                        step = 2),
                padding='same',
                activation='relu'))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.5))
    keras_model.add(Conv1D(filters=hp.Int('filters',
                                        min_value=32,
                                        max_value=512,
                                        step = 32),
                kernel_size=hp.Int('kernel_size',
                                        min_value=3,
                                        max_value=11,
                                        step = 2),
                padding='same',
                activation='relu'))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.5))
    keras_model.add(Conv1D(filters=hp.Int('filters',
                                        min_value=32,
                                        max_value=512,
                                        step = 32),
                kernel_size=hp.Int('kernel_size',
                                        min_value=3,
                                        max_value=11,
                                        step = 2),
                padding='same',
                activation='relu'))
    keras_model.add(GlobalMaxPool1D())
    keras_model.add(layers.Flatten())
    keras_model.add(Dropout(0.5))
    keras_model.add(Dense(units=hp.Int('units',
                                        min_value=64,
                                        max_value=256,
                                        step=32),
                           activation='relu'))
    keras_model.add(Dropout(0.5))
    keras_model.add(Dense(2))
    keras_model.add(Activation('softmax'))
    keras_model.compile(optimizer=keras.optimizers.Adam(
        hp.Choice('learning_rate',
                  values=[1e-2, 1e-3, 1e-4])),
    loss='binary_crossentropy',
    metrics=['accuracy'])
    return keras_model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    executions_per_trial=3,
    directory='/content/drive/MyDrive/Dincy/CNN_tuner',
    project_name='1042')
tuner.search_space_summary()

Search space summary
Default search space size: 4
filters (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
kernel_size (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 11, 'step': 2, 'sampling': None}
units (Int)
{'default': None, 'conditions': [], 'min_value': 64, 'max_value': 256, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
tuner.search(x_train, y_train,
             epochs=10,
             validation_data=(x_val, y_val))

models = tuner.get_best_models(num_models=2)

Trial 1 Complete [00h 12m 19s]
val_loss: 0.5588799317677816

Best val_loss So Far: 0.5588799317677816
Total elapsed time: 00h 12m 19s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
160               |288               |filters
7                 |3                 |kernel_size
64                |160               |units
0.0001            |0.0001            |learning_rate

Epoch 1/10
1289/1289 [==============================] - 23s 17ms/step - loss: 0.6994 - accuracy: 0.5497 - val_loss: 0.6901 - val_accuracy: 0.5655
Epoch 2/10
1289/1289 [==============================] - 21s 16ms/step - loss: 0.6854 - accuracy: 0.5643 - val_loss: 0.6777 - val_accuracy: 0.5655
Epoch 3/10
1289/1289 [==============================] - 20s 16ms/step - loss: 0.6676 - accuracy: 0.5754 - val_loss: 0.6575 - val_accuracy: 0.6144
Epoch 4/10
1289/1289 [==============================] - 21s 16ms/step - loss: 0.6459 - accuracy: 0.6110 - val_loss: 0.6331 - val_accuracy: 0.6443
Epoch 5/1

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size,
                        EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH,
                        trainable=traina))
model.add(Conv1D(288,3,padding='same',activation='relu'))
model.add(MaxPool1D())
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(160, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
keras_model.fit(x_train, y_train, batch_size=16, epochs=1,  validation_data=(x_val,y_val))

Epoch 1/50
905/905 [==============================] - 161s 175ms/step - loss: 0.1724 - accuracy: 0.9197 - val_loss: 0.0609 - val_accuracy: 0.9829
Epoch 2/50
905/905 [==============================] - 124s 137ms/step - loss: 0.0651 - accuracy: 0.9793 - val_loss: 0.0521 - val_accuracy: 0.9851
Epoch 3/50
905/905 [==============================] - 124s 137ms/step - loss: 0.0482 - accuracy: 0.9854 - val_loss: 0.0512 - val_accuracy: 0.9856
Epoch 4/50
905/905 [==============================] - 124s 137ms/step - loss: 0.0414 - accuracy: 0.9886 - val_loss: 0.0512 - val_accuracy: 0.9873
Epoch 5/50
905/905 [==============================] - 125s 138ms/step - loss: 0.0344 - accuracy: 0.9902 - val_loss: 0.0564 - val_accuracy: 0.9851
Epoch 6/50
905/905 [==============================] - 119s 132ms/step - loss: 0.0272 - accuracy: 0.9921 - val_loss: 0.0599 - val_accuracy: 0.9851
Epoch 7/50
905/905 [==============================] - 118s 131ms/step - loss: 0.0240 - accuracy: 0.9929 - val_loss: 0.0573 -

In [ ]:
y_test = np.argmax(y_test, axis=1)
y_pred_enc = model.predict(x_test)
y_pred_enc = np.argmax(y_pred_enc, axis=1)

169/169 [==============================] - 0s 1ms/step


In [ ]:
print(classification_report(y_test,y_pred_enc))

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
seed = 42
n_splits=10
np.random.seed(seed)
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)

In [ ]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
# K-fold Cross Validation model evaluation
fold_no = 1

In [ ]:
texts = np.array(text)

In [ ]:
y=df['isValid']
y=np.array(y)

In [ ]:
y

array([1, 1, 0, ..., 1, 0, 1])

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

In [ ]:
checkpointer = ModelCheckpoint('CNN' + "_weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor="val_loss", verbose=1,
                               save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', mode='auto')
tensorboard_cnn = TensorBoard(log_dir='./Graph_CNN', histogram_freq=0,write_graph=True, write_images=True)

In [ ]:
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, f1_score, auc,
                             plot_confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, 
                             precision_recall_curve)

In [ ]:
# Accuracy scores lists
acc_scores, prec_scores, recall_scores, f1_scores, fold = [], [], [], [], []
for train, test in kfold.split(texts, y):
  x_train=texts[train]
  y_train=y[train]
  x_test=texts[test]
  y_test=y[test]
  le = preprocessing.LabelEncoder()
  y_train = le.fit_transform(y_train)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test) 
  model = Sequential()
  model.add(Embedding(vocab_size,
                          EMBEDDING_DIM,
                          weights=[embedding_matrix],
                          input_length=MAX_SEQUENCE_LENGTH,
                          trainable=traina))
  model.add(Conv1D(160,7,padding='same',activation='relu'))
  model.add(MaxPool1D())
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(96, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(2, activation='sigmoid'))
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])
  model.fit(x_train, y_train, batch_size=16, epochs=10, validation_split=0.1)
  
  pred = model.predict(x_test)
  pred=np.argmax(pred, axis=1)
  ytest=np.argmax(y_test, axis=1)
  print(classification_report(ytest,pred))
  fig, ax = plt.subplots(figsize=(7, 7))
  sns.heatmap(confusion_matrix(ytest, pred), annot=True, 
              cbar=False, fmt='1d', cmap='Blues', ax=ax)
  ax.set_title('Confusion Matrix', loc='left', fontsize=16)
  ax.set_xlabel('Predicted')
  ax.set_ylabel('Actual')
  plt.show()
  #acc_scores=acc_scores.append(accuracy_score(ytest,pred))
  #f1_scores=f1_scores.append(f1_score(y_test,pred,average="weighted"))
  # Increase fold number
  fold_no = fold_no + 1
print("-"*30)
print(f'Accuracy:  {np.mean(acc_scores):.5f} +- {np.std(acc_scores):5f}')
print(f'F1 Score:  {np.mean(f1_scores):.5f} +- {np.std(f1_scores):5f}')
#print(f'ROC AUC:   {np.mean(roc_auc_scores):.5f} +- {np.std(roc_auc_scores):5f}')
print("")


In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')